In [42]:
!pip install transformers==4.0.1

  Using cached transformers-4.0.1-py3-none-any.whl (1.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.2.2
    Uninstalling transformers-4.2.2:
      Successfully uninstalled transformers-4.2.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

sentence-transformers 0.3.9 requires transformers<3.6.0,>=3.1.0, but you'll have transformers 4.0.1 which is incompatible.
You should consider upgrading via the '/home/ptah_old/anaconda3/envs/pyconda/bin/python -m pip install --upgrade pip' command.


In [43]:
from transformers import AutoTokenizer

In [44]:
from transformers import Trainer, TrainingArguments

In [45]:
import datasets

In [46]:
from datasets import load_dataset
datasetsx = load_dataset('wikitext', 'wikitext-2-raw-v1')

Reusing dataset wikitext (/home/ptah_old/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91)


In [47]:
datasetsx

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [48]:
import pandas as pd

In [49]:
winohack = pd.read_csv('combined_sentences.csv')

In [50]:
winohack.columns = ['sentences']

In [51]:
winohack = winohack.sample(20000)

In [52]:
winohack_dataset = datasets.Dataset.from_pandas(winohack)

In [53]:
winohack_dataset

Dataset({
    features: ['sentences', '__index_level_0__'],
    num_rows: 20000
})

In [54]:
from sklearn.model_selection import train_test_split

In [55]:
train_data, val_data = train_test_split(winohack, test_size = 0.2 )

In [56]:
model_checkpoint = "bert-base-uncased"

In [57]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [58]:
tokenizer.mask_token_id

103

In [59]:
import numpy as np

In [60]:
def regular_encode(texts, tokenizer, maxlen=25):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        padding = 'max_length',
        max_length = 24
    )
    return enc_di

In [61]:
X_train = regular_encode(list(train_data.sentences.values), tokenizer, maxlen=25)
X_val = regular_encode(list(val_data.sentences.values), tokenizer, maxlen=25)

In [62]:
X_train = datasets.Dataset.from_dict(X_train)
X_val = datasets.Dataset.from_dict(X_val)

In [63]:
# def tokenize_function(examples):
#     return tokenizer(examples["sentences"])

In [64]:
#tokenized_datasets = winohack_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=['sentences', '__index_level_0__'])

In [65]:
#block_size = 25

In [66]:
#test = {k: sum(tokenized_datasets[1][k], []) for k in tokenized_datasets[1].keys()}

In [67]:
# def group_texts(examples):
#     # Concatenate all texts.
#     concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
#     print(concatenated_examples)
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     print(total_length)
#     # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
#         # customize this part to your needs.
#     total_length = (total_length // block_size) * block_size
#     # Split by chunks of max_len.
#     result = {
#         k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

In [68]:
# lm_datasets = tokenized_datasets.map(
#     group_texts,
#     batched=True,
#     batch_size=1000,
#     num_proc=4,
# )

In [69]:
#testem = np.random.rand(*X_train.shape) < 0.15

In [70]:
# def prepare_mlm_input_and_labels(X):
#     # 15% BERT masking
#     inp_mask = np.random.rand(*X.shape)<0.15 
#     # do not mask special tokens
#     inp_mask[X<=2] = False
#     # set targets to -1 by default, it means ignore
#     labels =  -1 * np.ones(X.shape, dtype=int)
#     # set labels for masked tokens
#     labels[inp_mask] = X[inp_mask]
    
#     # prepare input
#     X_mlm = np.copy(X)
#     # set input to [MASK] which is the last token for the 90% of tokens
#     # this means leaving 10% unchanged
#     inp_mask_2mask = inp_mask  & (np.random.rand(*X.shape)<0.90)
#     X_mlm[inp_mask_2mask] = tokenizer  # mask token is the last in the dict

#     # set 10% to a random token
#     inp_mask_2random = inp_mask_2mask  & (np.random.rand(*X.shape) < 1/9)
#     X_mlm[inp_mask_2random] = np.random.randint(3, 250001, inp_mask_2random.sum())
    
#     return X_mlm, labels

In [71]:
#X_train_mlm, y_train_mlm = prepare_mlm_input_and_labels(X_train_mlm)

In [72]:
from transformers import AutoModelForMaskedLM

In [73]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [74]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, mlm=True)

In [76]:
training_args = TrainingArguments(
    "test-mlm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [77]:
train_data

,sentences
559671,The inspector called the owner because she had...
1012425,The librarian asked the bystander to provide r...
1171987,The visitor called the janitor to get an estim...
1697765,The resident gave the electrician feedback on ...
2360238,The paramedic told the pedestrian that she wou...
...,...
2225516,The surgeon had to rescue the taxpayer from th...
1848312,The teacher informed the employee that her cat...
1921760,The lawyer gave the patient a laser cutter dem...
1610695,The specialist arrested the teenager even thou...


In [79]:
!pip install docker

/home/ptah_old/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     |████████████████████████████████| 146 kB 935 kB/s eta 0:00:01
     |████████████████████████████████| 200 kB 1.0 MB/s eta 0:00:01
You should consider upgrading via the '/home/ptah_old/anaconda3/envs/pyconda/bin/python -m pip install --upgrade pip' command.


In [80]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_train,
    eval_dataset=X_val,
    data_collator=data_collator,
)

In [81]:
trainer.train()

/home/ptah_old/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 